In [1]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q
from pandasticsearch import Select, DataFrame

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# Notebook - Kabana lookalike

In [3]:
from IPython.display import Image
image="https://i0.wp.com/sisypheanhigh.com/foot/wp-content/uploads/2015/08/sisyphus-red.png"
Image(url=image)

In [4]:
hostname="http://noname.us.cray.com:30200"
client =  Elasticsearch(hostname, http_compress=True)

In [5]:
cluster_health=client.cluster.health()
#cluster_health

In [6]:
client_info=client.info()

## Read the data from elasticsearch

__Note:__ We use wildcard on the shasta-logs, so we are pulling from _ALL_ data sources

In [38]:
resp = client.search(
    index="shasta-logs-*",
    body={
        "size":0,
        "query": {
            "bool": {
              "must": [{
                  "match_all": {}
                },
                {
                  "range": {
                    "timereported": {
                      "gte": 1563847200000,
                      "lte": 1563978841842,
                      "format": "epoch_millis"
                }
              }
            }
          ],
          "must_not": []
        }
      },
        "_source":{
            "excludes":[]
        },
        "aggs": {
        "2": {
          "date_histogram": {
            "field": "timereported",
            "interval": "30s",
            "time_zone": "America/Chicago",
            "min_doc_count": 1
          }
        }
      }
    }
)

In [20]:
print("Number of responses:%d." % resp['hits']['total'])

Number of responses:7822822.


In [23]:
table=resp['aggregations']['2']['buckets']

In [24]:
pd_table=pd.DataFrame.from_dict(table)
pd_table.drop(columns="key", inplace=True)
#pd_table.drop(columns="key_as_string", inplace=True)
pd_table.head(5)

,key_as_string,doc_count
0,2019-07-22T22:20:30.000-05:00,587
1,2019-07-22T22:21:00.000-05:00,333
2,2019-07-22T22:21:30.000-05:00,316
3,2019-07-22T22:22:00.000-05:00,74
4,2019-07-22T22:22:30.000-05:00,639


Generate some statistical report on the message counts.

In [39]:
pd_table['doc_count'].describe()

count     4228.000000
mean      1850.241722
std       1578.007346
min         64.000000
25%       1177.000000
50%       1279.000000
75%       2693.250000
max      42986.000000
Name: doc_count, dtype: float64

In [11]:
#pd_table.columns

In [12]:
#pd_table.shape

In [13]:
#pd_table.hist(column='doc_count', color='#86bf91',grid=False,bins=100)

In [28]:
# Bokeh libraries
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel
from bokeh.models.tools import HoverTool
from bokeh.models import DatetimeTickFormatter
from math import pi

In [30]:
output_notebook()  # Render inline in a Jupyter Notebook

Loading BokehJS ...

In [31]:
#print(pd_table.head())
pd_table['key_as_string']=pd.to_datetime(pd_table['key_as_string'].astype(str), format='%Y-%m-%dT%H:%M:%S.%f%z')
#print(pd_table.head())

In [32]:
#pd_table.columns.tolist()

In [33]:
sample=pd_table.sample(100)
X=sample['key_as_string'].tolist()
Y=sample['doc_count'].tolist()

In [37]:
p = figure(x_axis_type="datetime",plot_width=1200, plot_height=400)
source = ColumnDataSource(data={
    'datetime' : X,
    'count' : Y,
})
#p.circle(X, Y, size = 5, color = 'green')
p.circle(x='datetime',y='count', source=source, size = 5, color = 'green' )

p.title.text = 'Message Counts per 30 minutes'
p.background_fill_color="#f5f5f5"
p.grid.grid_line_color="white"
p.yaxis.axis_label = 'Count'
p.xaxis.axis_label =' timereported per 30 minutes'
p.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )
p.xaxis.major_label_orientation = pi/4

hover = HoverTool(
    tooltips=[
        ('Count', '@count{%d}'),
        ('timereported per 30 seconds','@datetime{%Y-%m-%d %H:%M:%S.%3N}')
    ],
    formatters={
        'count':'printf',
        'datetime': 'datetime',
    },
    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline'
)

p.add_tools(hover)
show(p)